In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import math
import numpy as np
from tqdm import tqdm

In [2]:
torch.manual_seed(42)  # this is to ensure deterministic behavior

In [3]:
def positional_encoding_function(maxL, D):
        pe = torch.zeros(maxL, D)
            # pe measures (maxL, D)
        position = torch.arange(0, maxL).unsqueeze(1)
            # (maxL) --> (maxL, 1) via unsqueeze(1)
        coeff = torch.exp(torch.arange(0, D, 2).float() * -(math.log(10000.0) / D))

        pe[:, 0::2] = torch.sin(position * coeff)  # fill in even positions
        pe[:, 1::2] = torch.cos(position * coeff)  # fill in odd positions
        return pe

In [4]:
positional_encoding_function(10, 4).shape

torch.Size([10, 4])

In [5]:

class PositionalEncoding(nn.Module):
    def __init__(self, D, maxL):
        super(PositionalEncoding, self).__init__()
        self.pe = nn.Parameter(positional_encoding_function(maxL, D).unsqueeze(0), requires_grad=False) # FIXME
    def forward(self, x):
        return x  + self.pe[:,:x.size(1)]# FIXME

In [7]:
class FeedForward(nn.Module):
    def __init__(self, D, D_ff):
        super(FeedForward, self).__init__()
        self.fc1 = nn.Linear(D,D_ff) # FIXME
        self.fc2 = nn.Linear(D_ff, D) # FIXME
        self.relu = nn.ReLU() # FIXME

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        return self.fc2(x) # FIXME

In [27]:
def split_heads(input, H):  # tensor shape is [B, L, D]
    Dh = input.shape[2] // H
    B, L, D = input.shape
    input = input.view(B, L, H, Dh)
    return input.permute(0, 2, 1, 3).contiguous() # FIXME

def combine_heads(input):  # tensor shape is [B, H, L, D]
    B, H, L, D = input.shape
    input = input.permute(0, 2, 1, 3).contiguous()
    return input.view(B, L, H * D)
     # FIXME

In [29]:
def generate_causality_mask(target):
    L = target.size(1)
    return torch.tril(torch.ones(1, L, L), diagonal=0).int()

In [73]:
def scaled_dot_product_attention(Q, K, V, d_k, mask=None, negative_infinity=-1e9):
    # print(Q,K)
    attn_scores = torch.matmul(Q,K.transpose(-2,-1)) / (d_k**0.5) # FIXME (step 1)
    if mask is not None:
      attn_scores = torch.where(mask == 0, negative_infinity,attn_scores) # FIXME (step 2)
    attn_probabilities = nn.Softmax(dim=-1)
    attn_probabilities = attn_probabilities(attn_scores)
    output = torch.matmul(attn_probabilities, V) # FIXME (step 4)
    return output

In [75]:

class MultiHeadAttention(nn.Module):
    def __init__(self, D, H):
        super(MultiHeadAttention, self).__init__()
        assert D % H == 0, "D must be divisible by H"
        self.D = D
        self.H = H
        self.Dh = D//H
        self.W_q = nn.Linear(D, D)
        self.W_k = nn.Linear(D, D)
        self.W_v = nn.Linear(D, D)
        self.W_o = nn.Linear(D, D)
    def forward(self, Q, K, V, mask=None):
        Q = split_heads(self.W_q(Q), self.H)
        K = split_heads(self.W_k(K), self.H)
        V = split_heads(self.W_v(V), self.H)

        output = scaled_dot_product_attention(Q, K, V, self.Dh, mask, negative_infinity=-1e9)
        output = combine_heads(output)

        output = self.W_o(output)
        return output # FIXME

In [77]:

class EncoderLayer(nn.Module):
    def __init__(self, D, H, D_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attention = MultiHeadAttention(D,H) # FIXME
        self.feed_forward =  FeedForward(D, D_ff)# FIXME
        self.norm1 = nn.LayerNorm(D) # FIXME
        self.norm2 = nn.LayerNorm(D) # FIXME
        self.dropout = nn.Dropout(dropout) # FIXME

    def forward(self, x, mask=None):
        attention = self.self_attention(x, x, x, mask)
        x = self.norm1(x + self.dropout(attention))
        ff = self.feed_forward(x)

        return self.norm2(x + self.dropout(ff)) # FIXME


In [79]:

class DecoderLayer(nn.Module):
    def __init__(self, D, H, D_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attention = MultiHeadAttention(D, H) # FIXME
        self.cross_attention = MultiHeadAttention(D, H) # FIXME
        self.feed_forward = FeedForward(D, D_ff) # FIXME
        self.norm1 = nn.LayerNorm(D) # FIXME
        self.norm2 = nn.LayerNorm(D) # FIXME
        self.norm3 = nn.LayerNorm(D) # FIXME
        self.dropout = nn.Dropout(dropout) # FIXME

    def forward(self, x, encoder_output, self_mask, cross_mask=None):
        self_attention = self.self_attention(x, x, x, self_mask)
        x = self.norm1(x + self.dropout(self_attention))
        cross_attention = self.cross_attention(x, encoder_output, encoder_output, cross_mask)
        x = self.norm2(x + self.dropout(cross_attention))
        ff = self.feed_forward(x)
        return self.norm3(x + self.dropout(ff))


In [81]:

class Transformer(nn.Module):
    def __init__(self, source_vocab_size, target_vocab_size, D,
                H, Nx, D_ff, maxL, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(source_vocab_size, D) # FIXME
        self.decoder_embedding = nn.Embedding(target_vocab_size, D) # FIXME
        self.positional_encoding = PositionalEncoding(D, maxL) # FIXME
        self.encoder_layers = nn.ModuleList([EncoderLayer(D, H, D_ff, dropout) for _ in range(Nx)]) # FIXME
        self.decoder_layers = nn.ModuleList([DecoderLayer(D, H, D_ff, dropout) for _ in range(Nx)]) # FIXME
        self.fc = nn.Linear(D, target_vocab_size) # FIXME
        self.dropout = nn.Dropout(dropout) # FIXME

    def forward(self, source, target):
        source = self.positional_encoding(self.dropout(self.encoder_embedding(source)))
        target = self.positional_encoding(self.dropout(self.decoder_embedding(target)))
        causality_mask = generate_causality_mask(target)
        encoder_output = source
        for encoder_layer in self.encoder_layers:
            encoder_output = encoder_layer(encoder_output)
        decoder_output = target
        for decoder_layer in self.decoder_layers:
            decoder_output = decoder_layer(decoder_output, encoder_output, causality_mask)
        output = self.fc(decoder_output)
        return output


In [68]:
source_vocab_size = 5000
target_vocab_size = 5000
D = 48
H = 3
Nx = 3
D_ff = 128
maxL = 20
dropout = 0.1

transformer = Transformer(source_vocab_size, target_vocab_size,
                         D, H, Nx, D_ff, maxL, dropout)

In [69]:
B = 64
Ls = maxL  # arbitrary, but must be > 0 and <= maxL
Lt = maxL  # arbitrary, but must be > 0 and <= maxL
source_data = torch.randint(1, source_vocab_size, (B, Ls))
target_data = torch.randint(1, target_vocab_size, (B, Lt))

# generate a [B, L] tensor whose values are [1, source_vocab_size)
# generate a [B, L] tensor whose values are [1, target_vocab_size)

In [70]:
criterion = nn.CrossEntropyLoss(ignore_index=0)


optimizer = optim.Adam(transformer.parameters(),
                      lr=1e-4, betas=(0.9, 0.98), eps=1e-9)

transformer.train()
for epoch in tqdm(range(5001)):
    optimizer.zero_grad()
    output = transformer(source_data, target_data[:, :-1])
    loss = criterion(output.reshape(
        -1, target_vocab_size), target_data[:, 1:].reshape(-1))
    loss.backward()
    optimizer.step()
    if epoch % 50 == 0:
        tqdm.write(f'Epoch: {epoch+1}, Loss: {loss.item()})')

  0%|          | 2/5001 [00:00<18:32,  4.49it/s]

Epoch: 1, Loss: 8.70529842376709)


  1%|          | 52/5001 [00:09<13:19,  6.19it/s]

Epoch: 51, Loss: 8.294711112976074)


  2%|▏         | 102/5001 [00:18<15:07,  5.40it/s]

...

Epoch: 4901, Loss: 0.0004795412824023515)


 99%|█████████▉| 4952/5001 [13:51<00:07,  6.16it/s]

Epoch: 4951, Loss: 0.00048454420175403357)


100%|██████████| 5001/5001 [14:00<00:00,  5.95it/s]

Epoch: 5001, Loss: 0.000415522517869249)
